In [50]:
# %% Build and save tessellation GRAPH edges to GPKG
import geopandas as gpd
from shapely.geometry import LineString
from libpysal.graph import read_parquet as read_graph

input_datadir = "D:/Work/Github_Morphotopes/data/"
region_id = 0

tess_path  = input_datadir + f"tessellations/tessellation_{region_id}.parquet"
graph_path = input_datadir + f"neigh_graphs/tessellation_graph_{region_id}.parquet"
gpkg_file  = input_datadir + f"neigh_graphs/tess_graph_{region_id}.gpkg"

# 1) load base tessellation + graph
gdf = gpd.read_parquet(tess_path)
G   = read_graph(graph_path)  # <-- THIS reads your tessellation graph parquet

# 2) neighbors & graph node labels
W = G.to_W()
neighbors, weights = W.neighbors, W.weights
try:
    ids = W.id_order
except AttributeError:
    ids = list(W.neighbors.keys())

# representative points & align to graph labels
pts = gdf.geometry.representative_point()
pts_aligned = pts.reindex(ids)
if pts_aligned.isna().any():
    raise RuntimeError("Graph node IDs don't match tessellation index. Reload the original parquet.")

# 3) build undirected edge set
rank = {lab: k for k, lab in enumerate(ids)}
edge_geoms, src_idx, dst_idx, edge_w = [], [], [], []
for i, nbrs in neighbors.items():
    for j, w in zip(nbrs, weights[i]):
        if rank[i] < rank[j]:  # keep one direction
            edge_geoms.append(LineString([pts_aligned.loc[i], pts_aligned.loc[j]]))
            src_idx.append(i); dst_idx.append(j); edge_w.append(w)

edges = gpd.GeoDataFrame(
    {"src": src_idx, "dst": dst_idx, "weight": edge_w},
    geometry=edge_geoms, crs=gdf.crs
)

# 4) write GPKG (base first, then edges). Use pyogrio; start clean with mode="w"
# If CRS is missing, set your EPSG (e.g., 25832)
if gdf.crs is None:
    gdf = gdf.set_crs(25832)
edges = edges.set_crs(gdf.crs)

# keep only geometry to avoid schema issues
base = gdf.loc[gdf.geometry.notna() & ~gdf.geometry.is_empty, ["geometry"]]
edges = edges.loc[edges.geometry.notna() & ~edges.geometry.is_empty]

base.to_file(gpkg_file, layer="tessellation_base", driver="GPKG", mode="w", engine="pyogrio", index=False)
edges.to_file(gpkg_file, layer="graph_edges",       driver="GPKG", mode="a", engine="pyogrio", index=False)

print("Open in QGIS:", gpkg_file, "(layers: tessellation_base, graph_edges)")


Open in QGIS: D:/Work/Github_Morphotopes/data/neigh_graphs/tess_graph_0.gpkg (layers: tessellation_base, graph_edges)


In [51]:
# %% Visualize multiple neighbor graphs -> GPKG files in neigh_graphs/
import os, re
import geopandas as gpd
from shapely.geometry import LineString
from libpysal.graph import read_parquet as read_graph

# ---- folders (your style) ----
input_datadir     = "D:/Work/Github_Morphotopes/data/"
neigh_graphs_dir  = input_datadir + "neigh_graphs/"
tess_dir          = input_datadir + "tessellations/"
bldg_dir          = input_datadir 
encl_dir          = input_datadir + "enclosures/"
streets_dir       = input_datadir + "streets/"

# Which graph types to export (pick any subset)
graph_types = ["tessellation", "building", "enclosure", "street"]  # add "nodes" if you want

# Region IDs to process — either set explicitly, or auto-discover from graph files
region_ids = []
if not region_ids:
    # discover region IDs that have graph files for the first type present
    import glob
    pat = neigh_graphs_dir + f"{graph_types[0]}_graph_*.parquet"
    for p in glob.glob(pat):
        m = re.search(r"_(\d+)\.parquet$", p)
        if m:
            region_ids.append(int(m.group(1)))
region_ids = sorted(set(region_ids)) or [0]  # fallback to [0]

def base_and_graph_paths(gtype: str, rid: int):
    if gtype == "tessellation":
        base = tess_dir    + f"tessellation_{rid}.parquet"
        graph = neigh_graphs_dir + f"tessellation_graph_{rid}.parquet"
    elif gtype == "building":
        base = bldg_dir    + f"buildings_{rid}.parquet"
        graph = neigh_graphs_dir + f"building_graph_{rid}.parquet"
    elif gtype == "enclosure":
        base = encl_dir    + f"enclosure_{rid}.parquet"
        graph = neigh_graphs_dir + f"enclosure_graph_{rid}.parquet"
    elif gtype == "street":
        base = streets_dir + f"streets_{rid}.parquet"
        graph = neigh_graphs_dir + f"street_graph_{rid}.parquet"
    elif gtype == "nodes":
        # special case; needs node points from streets via momepy (not shown here)
        base = streets_dir + f"streets_{rid}.parquet"
        graph = neigh_graphs_dir + f"nodes_graph_{rid}.parquet"
    else:
        raise ValueError("Unknown graph type")
    return base, graph

def build_edges_from_graph(base_gdf: gpd.GeoDataFrame, graph):
    """Return a GeoDataFrame of LineStrings between neighbor pairs (undirected)."""
    # 1) label points to draw edges (use representative_point for robustness)
    pts = base_gdf.geometry.representative_point()

    # 2) neighbors + label alignment
    W = graph.to_W()
    neighbors, weights = W.neighbors, W.weights
    try:
        ids = W.id_order
    except AttributeError:
        ids = list(neighbors.keys())

    # align by labels (index of base_gdf must match what graph was built from)
    pts_aligned = pts.reindex(ids)
    if pts_aligned.isna().any():
        raise RuntimeError("Graph node IDs don't match the base GeoDataFrame index. "
                           "Reload the original base parquet used to build the graph.")

    rank = {lab: k for k, lab in enumerate(ids)}  # for i<j dedup
    geoms, src, dst, wts = [], [], [], []
    for i, nbrs in neighbors.items():
        wi = weights[i]
        for j, w in zip(nbrs, wi):
            if rank[i] < rank[j]:
                geoms.append(LineString([pts_aligned.loc[i], pts_aligned.loc[j]]))
                src.append(i); dst.append(j); wts.append(w)

    return gpd.GeoDataFrame({"src": src, "dst": dst, "weight": wts},
                            geometry=geoms, crs=base_gdf.crs)

# ---- loop and export ----
for rid in region_ids:
    for gtype in graph_types:
        base_path, graph_path = base_and_graph_paths(gtype, rid)
        if not (os.path.exists(base_path) and os.path.exists(graph_path)):
            print(f"SKIP {gtype} {rid}: missing base or graph parquet")
            continue

        base = gpd.read_parquet(base_path)
        G = read_graph(graph_path)

        # build edge lines
        edges = build_edges_from_graph(base, G)

        # ensure CRS + drop empties
        if base.crs is None:
            base = base.set_crs(25832)  # change if your data uses a different EPSG
        edges = edges.set_crs(base.crs)
        base_clean  = base.loc[base.geometry.notna()  & ~base.geometry.is_empty,  ["geometry"]]
        edges_clean = edges.loc[edges.geometry.notna() & ~edges.geometry.is_empty, ["geometry","src","dst","weight"]]

        # write: one GPKG per graph, placed in neigh_graphs/
        out_gpkg = neigh_graphs_dir + f"viz_{gtype}_graph_{rid}.gpkg"

        # write base first (mode='w'), then append edges (mode='a'); prefer pyogrio on Windows
        try:
            base_clean.to_file(out_gpkg, layer="base", driver="GPKG", mode="w", engine="pyogrio", index=False)
            edges_clean.to_file(out_gpkg, layer="graph_edges", driver="GPKG", mode="a", engine="pyogrio", index=False)
        except Exception:
            # fallback to fiona if pyogrio isn't available
            if os.path.exists(out_gpkg):
                os.remove(out_gpkg)
            base_clean.to_file(out_gpkg, layer="base", driver="GPKG", mode="w", engine="fiona", index=False)
            edges_clean.to_file(out_gpkg, layer="graph_edges", driver="GPKG", mode="a", engine="fiona", index=False)

        print(f"Wrote {out_gpkg}  (layers: base, graph_edges)")


Wrote D:/Work/Github_Morphotopes/data/neigh_graphs/viz_tessellation_graph_0.gpkg  (layers: base, graph_edges)
Wrote D:/Work/Github_Morphotopes/data/neigh_graphs/viz_building_graph_0.gpkg  (layers: base, graph_edges)
Wrote D:/Work/Github_Morphotopes/data/neigh_graphs/viz_enclosure_graph_0.gpkg  (layers: base, graph_edges)
Wrote D:/Work/Github_Morphotopes/data/neigh_graphs/viz_street_graph_0.gpkg  (layers: base, graph_edges)
